![DSB logo](img/Dolan.jpg)
# Groupby and Pivoting

## PD4E Chapter 10: Groupby Operations: Split-Apply-Combine
### How do you read/manipulate/store data in Python?

# What You Learned in Python/Pandas that could Apply Here

You will need following knowledge from the first half of this course:
1. functions
2. subsetting/slicing data

# What You will Learn in this Chapter

You will learn following techniques in this chapter:
1. Groupby operations to aggregate, transform, and filter data
2. Built-in and custom user function to perform groupby functions
3. creating _pivot tables_ from DataFrames

# What do we mean by Groupby operations?

- Groupby operations is how we aggregate, transform and filter data
    - you will learn the `SQL GROUPBY` in BA 510
- We follow the mantra below:
    - data is separated into different parts based on their feature(s);
    - we apply applicable function(s) to different parts of the data;
    - then we combine different parts of processed data back as the result.

# Why do we need to do this?

- we may need to select subsets of a DataFrame using certain conditions
    - like how we calculate the `final_pay` based on `work_load` last week
- or in data analytics
    - we have different standardized processing pipelines for _categorical_ and _continuous_ features

# Aggregation

- Aggregation is a term we use in databases, or particularly data warehouses
    - aggregation also known as summarization, it means data reduction
    - most of descriptive stats, such as count, mean, standard deviation, are summarization methods
    - the data size is essentially smaller

# Aggregation Examples

- it refers to moving from a more specific data to a more abstract level
    - e.g., if the actual data is daily-based, and we want to get the monthly sum/average of the data
    - or if the observation are scattered in different years, and we want to look at the annual averages

In [1]:
# an example of aggregation
# load the gapminder data
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/gapminder.tsv'`
import pandas as pd
df = pd.read_csv('/srv/data/my_shared_data_folder/ba505-data/gapminder.tsv', sep='\t')
df.head(3)

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710


In [6]:
# calculate the annual average life expectancy for each year
df.groupby('year')['lifeExp'].mean()

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [9]:
# this is the longer alternative
df['year'].unique()

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007])

In [10]:
df1952 = df[df['year'] == 1952]
df1952mean = df1952['lifeExp'].mean()
df1952mean

49.057619718309866

# Built-in Aggregation Methods

- In above example, you noticed that we calculate the average life expactacies for each individual year
    - average is one of the __aggregation method__
    - there are other aggregation methods available
    - refer to PD4E pp. 192 for all applicable aggregation methods

In [12]:
# note that ball  columns in the 
# results below can be aggregation methods
df.groupby('continent')['lifeExp'].describe()

,count,mean,std,min,25%,50%,75%,max
continent,,,,,,,,
Africa,624.0,48.865330,9.150210,23.599,42.37250,47.7920,54.41150,76.442
Americas,300.0,64.658737,9.345088,37.579,58.41000,67.0480,71.69950,80.653
Asia,396.0,60.064903,11.864532,28.801,51.42625,61.7915,69.50525,82.603
Europe,360.0,71.903686,5.433178,43.585,69.57000,72.2410,75.45050,81.757
Oceania,24.0,74.326208,3.795611,69.120,71.20500,73.6650,77.55250,81.235


# Aggregation Methods

- So far we have been using aggregation methods directly
- but in general we can use the `.agg()` method for any aggregation function
    - you should consider `.agg()` as a special case of `.apply()`
- there are two use cases for using `.agg()`
    - we can only use `pandas` built-in aggregation methods (column 1 in Table 10.1) directly, but sometimes we prefer using the `numpy` alternatives (column 2 in Table 10.1) - since they are much faster
    - we can even use custom aggregation functions/methods

In [19]:
# numpy method
%time
import numpy as np
df.groupby('continent')['lifeExp'].agg(np.mean)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


continent
Africa      48.865330
Americas    64.658737
Asia        60.064903
Europe      71.903686
Oceania     74.326208
Name: lifeExp, dtype: float64

In [20]:
# custom aggregation function
def my_mean(values): # our own version of mean calculation
    # get total number of values
    return sum(values)/len(values)

In [21]:
df.groupby('continent')['lifeExp'].agg(my_mean)

continent
Africa      48.865330
Americas    64.658737
Asia        60.064903
Europe      71.903686
Oceania     74.326208
Name: lifeExp, dtype: float64

In [24]:
# multiple aggregation function siultaneously
df.groupby('continent')['lifeExp'].agg([np.count_nonzero, 
                                        np.mean, np.std, np.min, np.max])

,count_nonzero,mean,std,amin,amax
continent,,,,,
Africa,624.0,48.865330,9.150210,23.599,76.442
Americas,300.0,64.658737,9.345088,37.579,80.653
Asia,396.0,60.064903,11.864532,28.801,82.603
Europe,360.0,71.903686,5.433178,43.585,81.757
Oceania,24.0,74.326208,3.795611,69.120,81.235


In [25]:
df.groupby('continent')['lifeExp'].agg([np.count_nonzero, 
                                        np.mean, np.std]).rename(columns={'count_nonzero': 'count',
                                                                          'mean':'avg',
                                                                          'std':'std_dev'})

,count,avg,std_dev
continent,,,
Africa,624.0,48.865330,9.150210
Americas,300.0,64.658737,9.345088
Asia,396.0,60.064903,11.864532
Europe,360.0,71.903686,5.433178
Oceania,24.0,74.326208,3.795611


# Preferred `.groupby()` syntax

- You should use following syntax when you use `.groupby()`
    - the preferred syntax can handle different aggregation functions on different columns
    
```python
df.groupby('grouping column').agg({'aggregating column': 'aggregating function'})
```

In [26]:
df.groupby('continent').agg({'lifeExp': ['min', 'max'],
                              'pop': ['min', 'max']
                              }).round(0).head(10)

lifeExp            pop            
              min   max      min         max
continent                                   
Africa       24.0  76.0    60011   135031164
Americas     38.0  81.0   662850   301139947
Asia         29.0  83.0   120447  1318683096
Europe       44.0  82.0   147962    82400996
Oceania      69.0  81.0  1994794    20434176

In [27]:
df.groupby('continent').agg({'lifeExp': ['mean'],
                              'pop': ['mean']
                              }).round(0).head(10)

,lifeExp,pop
,mean,mean
continent,,
Africa,49.0,9916003.0
Americas,65.0,24504795.0
Asia,60.0,77038722.0
Europe,72.0,17169765.0
Oceania,74.0,8874672.0


# Transform Your Data

- Transform is different from aggregation
    - aggregate takes multiple values, and output one value (values --> annual average)
    - transform take mukltiple values, and do a one-to-one transform
    - transform is similar to apply

In [12]:
# z-score example
def my_zscore(x):
    return(x-x.mean()/x.std())

In [13]:
z_transform = df.groupby('continent')['lifeExp'].transform(my_zscore)
z_transform.head(3)

0    23.73844
1    25.26944
2    26.93444
Name: lifeExp, dtype: float64

In [14]:
# one-to-one match - same dimension with `df`
z_transform.shape[0] == df.shape[0]

True

In [15]:
# same as transform
z_transform1 = df.groupby('continent')['lifeExp'].apply(my_zscore)
z_transform1.head(3)

0    23.73844
1    25.26944
2    26.93444
Name: lifeExp, dtype: float64

# missing value example
- Read PD4E pp. 199 - 201 for the example

# Grouped

- You can always group your data based on one or more columns
- grouped objects are essentially subsets of your data, based on certain conditions

In [28]:
import seaborn as sns

tips10 = sns.load_dataset('tips').sample(10, random_state=2019)
tips10

,total_bill,tip,sex,smoker,day,time,size
4,24.59,3.61,Female,No,Sun,Dinner,4
42,13.94,3.06,Male,No,Sun,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
233,10.77,1.47,Male,No,Sat,Dinner,2
34,17.78,3.27,Male,No,Sat,Dinner,2
23,39.42,7.58,Male,No,Sat,Dinner,4
105,15.36,1.64,Male,Yes,Sat,Dinner,2
201,12.74,2.01,Female,Yes,Thur,Lunch,2
85,34.83,5.17,Female,No,Thur,Lunch,4
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [29]:
# this will return the grouped objects
grouped = tips10.groupby('sex')
grouped

In [31]:
# to unpack the grouped objects, we need to use a loop
for group in grouped:
    print(group)

('Male',      total_bill   tip   sex smoker  day    time  size
42        13.94  3.06  Male     No  Sun  Dinner     2
241       22.67  2.00  Male    Yes  Sat  Dinner     2
233       10.77  1.47  Male     No  Sat  Dinner     2
34        17.78  3.27  Male     No  Sat  Dinner     2
23        39.42  7.58  Male     No  Sat  Dinner     4
105       15.36  1.64  Male    Yes  Sat  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
4         24.59  3.61  Female     No   Sun  Dinner     4
201       12.74  2.01  Female    Yes  Thur   Lunch     2
85        34.83  5.17  Female     No  Thur   Lunch     4
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [33]:
# we can do calculation to the grouped objects
grouped.mean()

,total_bill,tip,size
sex,,,
Male,19.990,3.1700,2.333333
Female,21.885,3.4475,3.000000


In [37]:
# selecting a group - a subset of your data
grouped.get_group('Female')

,total_bill,tip,sex,smoker,day,time,size
4,24.59,3.61,Female,No,Sun,Dinner,4
201,12.74,2.01,Female,Yes,Thur,Lunch,2
85,34.83,5.17,Female,No,Thur,Lunch,4
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [38]:
# you can also group based on multiple columns
tips10.groupby(['sex','time']).mean().round(2)

total_bill   tip  size
sex    time                          
Male   Lunch          NaN   NaN   NaN
       Dinner       19.99  3.17  2.33
Female Lunch        23.78  3.59  3.00
       Dinner       19.98  3.30  3.00

# Multi-index DataFrame

- Above results, since it has two levels of indices, is called a _multi-index_ DataFrame
- multi-index DataFrames are useful in the field of databases and data warehouses/BI
- but strongly discouraged in machine learning

In [39]:
# indexing can unpack the multi-index, by giving you a subset of the data
tips10.groupby(['sex','time']).mean().loc['Male']

,total_bill,tip,size
time,,,
Lunch,NaN,NaN,NaN
Dinner,19.99,3.17,2.333333


In [40]:
# if you want to get all the data at the same level, you need `.reset_index()`
# this operation is called __flattening__
# almost a must do in machine learning
tips10.groupby(['sex','time']).mean().reset_index()

,sex,time,total_bill,tip,size
0,Male,Lunch,NaN,NaN,NaN
1,Male,Dinner,19.990,3.170,2.333333
2,Female,Lunch,23.785,3.590,3.000000
3,Female,Dinner,19.985,3.305,3.000000


# `pivot_table` in `pandas`

- Like you used in Excel, `pandas` also support pivot tables
- pivot tables are similar to `.groupby()`, but much cleaned
    - `NaN` rows will be eliminated
- Prefer `.pivot_table()` to `.groupby()` when you want to directly compare groups
- Alternatively, use `.groupby()` when you want to iterate through groups

In [41]:
tips10.pivot_table(index=['sex', 'time'],
                    values=['total_bill', 'tip', 'size'], aggfunc='mean').round(2)

size   tip  total_bill
sex    time                          
Male   Dinner  2.33  3.17       19.99
Female Lunch   3.00  3.59       23.78
       Dinner  3.00  3.30       19.98

# Crosstabs

- pandas also provide a function `crosstab()`
- only use that when you want to look at the relative frequency of a column values

In [42]:
# following statement will give you the relative frequency of `day` over `sex`
pd.crosstab(index=tips10['sex'], columns=tips10['day'], normalize='all').round(3)

day,Thur,Fri,Sat,Sun
sex,,,,
Male,0.0,0.0,0.5,0.1
Female,0.2,0.1,0.0,0.1


# Your Turn Here
Finish exercises below by following instructions of each of them.

## Q1. Coding Problem

Complete excecises regarding data types of the given DataFrame (`itinery_df`).

In [51]:
import random
import pandas as pd
# generating the DF
duration_mins = pd.Series(random.sample(range(1, 1800), 50), name='duration_mins')
work_types = ['lecture', 'consulting', 'research']
work_type_series = pd.Series(random.choices(work_types, k=50), name='work_types')
locations = ['Beijing, China', 'London, England', 'Paris, France', 'Munich, Germany', 
             'Sydney, Australia', 'Mumbai, India', 'Madrid, Spain']
loc_series = pd.Series(random.choices(locations, k=50), name='locations')
hour_rates = pd.Series([round(random.uniform(10.0, 20.0), 2) for i in range(50)], name='hour_rates')
#hour_rates.loc[random.sample(range(1, 20), 5)] = 'missing'
#duration_mins.loc[random.sample(range(1, 20), 5)] = 'missing'
itinery_df = pd.concat([duration_mins, work_type_series, loc_series, hour_rates], axis=1)
#itinery_df['duration_mins'] = itinery_df['duration_mins'].astype(str)
itinery_df.head()

,duration_mins,work_types,locations,hour_rates
0,1460,consulting,"Mumbai, India",17.78
1,1413,consulting,"Mumbai, India",16.70
2,238,research,"Munich, Germany",11.92
3,674,lecture,"Paris, France",12.76
4,372,lecture,"Paris, France",11.97


## Part 1:

Calculate the total working minutes (`durantion_mins`) by city, and answer which city has the highest total working minutes.

__HINT__: use `.groupby()` with the aggregation function as `sum`.

In [52]:
itinery_df.groupby('locations').agg({'duration_mins': ['sum']}).round(0).head(10)

,duration_mins
,sum
locations,
"Beijing, China",6664
"London, England",4279
"Madrid, Spain",1499
"Mumbai, India",11760
"Munich, Germany",4598
"Paris, France",7049
"Sydney, Australia",5052


## Part 2:

Without using the `.describe()` method, calculate the `count`, `min`, `max`, `std`, `mean` of `hour_rates` grouped by `work_types`.

In [53]:
itinery_df.groupby('work_types')['hour_rates'].describe()

,count,mean,std,min,25%,50%,75%,max
work_types,,,,,,,,
consulting,20.0,15.313500,3.181529,10.15,12.1100,16.145,17.7475,19.91
lecture,14.0,14.892857,2.424717,11.18,13.1750,14.180,17.4250,18.39
research,16.0,14.506250,2.301764,10.85,12.5425,14.650,16.0500,17.72


In [69]:
itinery_df.groupby('work_types')['hour_rates'].agg([np.count_nonzero, 
                                        np.mean, np.std, np.min, np.max])

,count_nonzero,mean,std,amin,amax
work_types,,,,,
consulting,20.0,15.313500,3.181529,10.15,19.91
lecture,14.0,14.892857,2.424717,11.18,18.39
research,16.0,14.506250,2.301764,10.85,17.72


## Part 3:

Print out subset of `itinery_df` by different `locations`

In [58]:
itinery_df['locations'].unique()

array(['Mumbai, India', 'Munich, Germany', 'Paris, France',
       'Madrid, Spain', 'Sydney, Australia', 'London, England',
       'Beijing, China'], dtype=object)

In [68]:
grouped = itinery_df.groupby('locations')
grouped
for group in grouped:
    print(group)

('Beijing, China',     duration_mins  work_types       locations  hour_rates
12            726    research  Beijing, China       11.50
13            765     lecture  Beijing, China       13.70
23           1367  consulting  Beijing, China       10.70
35           1397  consulting  Beijing, China       12.46
43            359     lecture  Beijing, China       11.18
44            993    research  Beijing, China       15.59
49           1057    research  Beijing, China       15.58)
('London, England',     duration_mins  work_types        locations  hour_rates
9            1276     lecture  London, England       18.39
11           1312  consulting  London, England       12.21
29              2  consulting  London, England       11.81
30           1225     lecture  London, England       14.97
34            363    research  London, England       13.13
38            101    research  London, England       16.38)
('Madrid, Spain',     duration_mins  work_types      locations  hour_rates
5      

## Part 4:

Use `.pivot_table` to compare which `location` has the highest `hour_rate`.

In [62]:
itinery_df.pivot_table(index=['locations'],
                    values=['hour_rates'], aggfunc='sum')

,hour_rates
locations,
"Beijing, China",90.71
"London, England",86.89
"Madrid, Spain",75.63
"Mumbai, India",177.45
"Munich, Germany",106.62
"Paris, France",120.51
"Sydney, Australia",89.06


## Part 5:

Show the relative frequencies of `work_types` over different `locations`. (__HINT__: use `crosstab()`)

In [66]:
pd.crosstab(index=itinery_df['work_types'], columns=itinery_df['locations'], normalize='all').round(5)

locations,"Beijing, China","London, England","Madrid, Spain","Mumbai, India","Munich, Germany","Paris, France","Sydney, Australia"
work_types,,,,,,,
consulting,0.04,0.04,0.04,0.10,0.06,0.06,0.06
lecture,0.04,0.04,0.04,0.04,0.04,0.08,0.00
research,0.06,0.04,0.02,0.08,0.04,0.04,0.04


# Classwork (start here in class)
You can start working on them right now:
- Read Chapter 10 in PD4E 
- If time permits, start in on your homework. 
- Ask questions when you need help. Use this time to get help from the professor!

# Homework (do at home)
The following is due before class next week:
  - Any remaining classwork from tonight
  - DataCamp “Bringing it all together” assignment
  - Coding assignment pt. 4

Note: All work on DataCamp is logged. Don't try to fake it!

Please email [me](mailto:jtao@fairfield.edu) if you have any problems or questions.

![DSB logo](img/Dolan.jpg)
# Groupby and Pivoting

## PD4E Chapter 10: Groupby Operations: Split-Apply-Combine
### How do you read/manipulate/store data in Python?